In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1,2"

import argparse
import json
import gc
import datasets
import transformers
import torch
import evaluate
from tqdm import tqdm
import json
import numpy as np
from trl import PPOConfig, PPOTrainer, AutoModelForSeq2SeqLMWithValueHead
from peft import LoraConfig, TaskType, get_peft_model, PeftModelForSeq2SeqLM, PeftModel
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

"""
Train fine tuned T5 model with Proximal Policy Optimization (PPO) algorithm.
"""
parser = argparse.ArgumentParser()
#parser.add_argument("--model_name", type=str, default="t5-small")
#parser.add_argument("--highlight", type=bool, default=True)
#args = parser.parse_args()


bertscore = evaluate.load("bertscore")
average_question_length = 10.0

HIGHLIGHT = True
TOKEN_QUESTION = "<question>"
TOKEN_END_QUESTION = "<question>"
TOKEN_CONTEXT = "<context>"
TOKEN_END_CONTEXT = "<context>"
TOKEN_ANSWER = "<answer>"
TOKEN_END_ANSWER = "<answer>"
HIGHLIGHT_ANSWER = "<hl>"
SPLIT_SEED = 42
NPROC = 32

model_name = "t5-small"
HIGHLIGHT = True
if HIGHLIGHT:
    model_name = f"{model_name}-hl"

/storagenfs/l.miglior/answer-aware-question-generation/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[2023-11-29 10:32:24,681] [INFO] [real_accelerator.py:158:get_accelerator] Setting ds_accelerator to cuda (auto detect)


/storagenfs/l.miglior/answer-aware-question-generation/.venv/lib/python3.10/site-packages/trl/trainer/ppo_config.py:141: UserWarning: The `optimize_cuda_cache` arguement will be deprecated soon, please use `optimize_device_cache` instead.
  warnings.warn(


In [2]:
peft_config = LoraConfig(
    r=128,
    lora_alpha=64,
    lora_dropout=0.1,
    target_modules=["q", "v"],
)

model = AutoModelForSeq2SeqLM.from_pretrained(f"./models/{model_name}/")
peft_model = PeftModel.from_pretrained(model, model_id=f"./models/{model_name}/", config=peft_config, device_map='auto')

In [3]:
ppo_model = AutoModelForSeq2SeqLMWithValueHead(peft_model)
ref_model = AutoModelForSeq2SeqLMWithValueHead(model)

tokenizer = transformers.AutoTokenizer.from_pretrained(f"./models/{model_name}", model_max_length=512)
torch.cuda.empty_cache()

In [17]:
def get_inputs_target(e):
    answer_start = e["answers"]["answer_start"][0]
    # add highlight token to context
    ans_len = len(e["answers"]["text"][0])

    if HIGHLIGHT:
        e["context"] = (
            e["context"][:answer_start]
            + " "
            + HIGHLIGHT_ANSWER
            + " "
            + e["context"][answer_start : answer_start + ans_len]
            + " "
            + HIGHLIGHT_ANSWER
            + " "
            + e["context"][answer_start + ans_len :]
        )

    return {
        # answer + context
        "inputs": f'generate question: {TOKEN_ANSWER} {e["answers"]["text"][0]} {TOKEN_END_ANSWER} {TOKEN_CONTEXT} {e["context"]} {TOKEN_END_CONTEXT}',
        # question
        "target": f'{TOKEN_QUESTION} {e["question"]} {TOKEN_END_QUESTION}',
    }


def preprocess_squad_dataset(dataset_name="squad", split="train"):
    dataset = datasets.load_dataset(dataset_name, split=split)
    # Add question, answer and context tokens to dataset in a new column named text
    dataset = dataset.map(
        lambda e: {
            # answer + context
            # changed to 'query' for PPO
            "query": f'generate question: {TOKEN_ANSWER} {e["answers"]["text"][0]} {TOKEN_END_ANSWER} {TOKEN_CONTEXT} {e["context"]} {TOKEN_END_CONTEXT}',
            # question
            "target": f'{TOKEN_QUESTION} {e["question"]} {TOKEN_END_QUESTION}',
        },
        num_proc=NPROC,
    )

    return dataset

# Need to have training dataset aligned with PPO input format

train_dataset = preprocess_squad_dataset(dataset_name="squad", split="train") 

In [37]:
def tokenize(sample):
    sample["input_ids"] = tokenizer.encode(sample["query"], return_tensors="pt", truncation=True, padding='max_length', max_length=512).squeeze()
    return sample

# Tokenize the dataset
train_dataset = train_dataset.map(tokenize, num_proc=NPROC)

Map (num_proc=32): 100%|██████████| 87599/87599 [00:07<00:00, 11379.79 examples/s]


In [41]:
train_dataset.select(range(1))['target']

['<question> To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France? <question>']

In [42]:
def reward_model(example):
    """
    this function will return a reward function for PPO
    """

    context = example["context"]
    question = example["question"]
    answer = example["answers"]["text"][0]

    reward = bertscore.compute(
        predictions=[prediction],
        references=[question],
        lang="en",
        model_type="bert-base-uncased",
    )["f1"].item()

    repetition_penalty = -1.0 if answer.lower() in prediction.lower() else 1.0
    question_word_penalty = -0.5 if question.split()[0].lower() != prediction.split()[0].lower() else 0.5
    ans_in_question_penalty = -1.0 if answer.lower() in question.lower() else 1.0
    question_length_penalty = -0.5 if len(question.split()) > average_question_length else 0.5

    reward += repetition_penalty + question_word_penalty + ans_in_question_penalty + question_length_penalty
    return reward

In [43]:
config = PPOConfig(
    learning_rate=1e-5,
    log_with='tensorboard',
    project_kwargs={'logging_dir': f'./logs/{model_name}'},
    batch_size=64,
)

ppo_trainer = PPOTrainer(
    model=ppo_model,
    ref_model=ref_model,
    config=config,
    dataset=train_dataset,
    tokenizer=tokenizer,
)

In [61]:
batch['prediction'][8]

'<pad> s s s s s s s s s '

In [62]:
for batch in tqdm(ppo_trainer.dataloader):
    query_tensors = batch["input_ids"]
    response_tensors = ppo_trainer.generate(query_tensors)
    batch["prediction"] = [tokenizer.decode(r.squeeze()) for r in response_tensors]
    print(batch["prediction"])
    pipe_outputs = reward_model(*zip(batch["query"], batch["prediction"]))
    rewards = [torch.tensor(output[1]["score"]) for output in pipe_outputs]

    stats = ppo_trainer.step(query_tensors, response_tensors, rewards)
    ppo_trainer.log_stats(stats, batch, rewards)
    
    print(f'objective/kl: {stats["objective/kl"]}')
    print(f'ppo/returns/mean: {stats["ppo/returns/mean"]}')
    print(f'ppo/policy/advantages_mean: {stats["ppo/policy/advantages_mean"]}')
    print('-'.join('' for x in range(100)))

  0%|          | 0/1368 [00:00<?, ?it/s]